In [2]:
from bs4 import BeautifulSoup
import requests
import re
from pymongo import MongoClient

In [6]:
APARTMENT = 'The Verge'

FLOORPLANS = [
    ('The Cusp', '17'),
    ('Greenway', '18')
]

results = []
for fp in FLOORPLANS:
    
    url = 'https://vergeapartments.com/floorplans/?action=check-availability&floorplan=' + fp[1]
    page_response = requests.get(url, timeout=5, headers={'X-Requested-With': 'XMLHttpRequest'})
    soup = BeautifulSoup(page_response.content, "html.parser")
    
    units = soup.find_all('tr', {'class': 'check-availability__row--lease-term'})
    for unit in units:
        data = {}
        data['apartment'] = APARTMENT
        data['floorplan'] = fp[0]
        data['unit'] = unit['data-unit']
        data['floor'] = -1
        
        leases = unit.find_all('label')
        for lease in leases:
            r = re.search('(\d{1,2}) Months months at \$(\d{4})\/mo', lease.text)
            length = r.group(1)
            price = int(r.group(2))
            
            data['lease_' + length] = price
        
        results.append(data)

results

[{'apartment': 'The Verge',
  'floorplan': 'The Cusp',
  'unit': '2380',
  'floor': -1,
  'lease_15': 2280,
  'lease_14': 2280,
  'lease_13': 2280,
  'lease_12': 2280,
  'lease_11': 2312,
  'lease_10': 2327,
  'lease_9': 2346,
  'lease_8': 2378,
  'lease_7': 2420,
  'lease_6': 2477,
  'lease_5': 3953,
  'lease_4': 2673,
  'lease_3': 2870},
 {'apartment': 'The Verge',
  'floorplan': 'The Cusp',
  'unit': '1466',
  'floor': -1,
  'lease_15': 2395,
  'lease_14': 2406,
  'lease_13': 2395,
  'lease_12': 2428,
  'lease_11': 2449,
  'lease_10': 2434,
  'lease_9': 2523,
  'lease_8': 2553,
  'lease_7': 2592,
  'lease_6': 2605,
  'lease_5': 2718,
  'lease_4': 2848,
  'lease_3': 5433},
 {'apartment': 'The Verge',
  'floorplan': 'The Cusp',
  'unit': '2639',
  'floor': -1,
  'lease_15': 2330,
  'lease_14': 2341,
  'lease_13': 2330,
  'lease_12': 2362,
  'lease_11': 2383,
  'lease_10': 2369,
  'lease_9': 2456,
  'lease_8': 2486,
  'lease_7': 2525,
  'lease_6': 2540,
  'lease_5': 2652,
  'lease_4': 

In [9]:
client = MongoClient('db', 27017, username='apartments', password='apartments')

db = client.apartments
db_units = db.units

db_result = db_units.insert_many(results)
print('Multiple posts: {0}'.format(db_result.inserted_ids))

Multiple posts: [ObjectId('5c6cdca8199a99002dc513d7'), ObjectId('5c6cdca8199a99002dc513d8'), ObjectId('5c6cdca8199a99002dc513d9'), ObjectId('5c6cdca8199a99002dc513da'), ObjectId('5c6cdca8199a99002dc513db'), ObjectId('5c6cdca8199a99002dc513dc'), ObjectId('5c6cdca8199a99002dc513dd'), ObjectId('5c6cdca8199a99002dc513de'), ObjectId('5c6cdca8199a99002dc513df'), ObjectId('5c6cdca8199a99002dc513e0')]


In [10]:
db_units.find_one({'unit': '2333'})

{'_id': ObjectId('5c6cdca8199a99002dc513e0'),
 'apartment': 'The Verge',
 'floorplan': 'Greenway',
 'unit': '2333',
 'floor': -1,
 'lease_15': 2489,
 'lease_14': 2501,
 'lease_13': 2489,
 'lease_12': 2524,
 'lease_11': 2547,
 'lease_10': 2532,
 'lease_9': 2626,
 'lease_8': 2658,
 'lease_7': 2701,
 'lease_6': 2718,
 'lease_5': 2840,
 'lease_4': 2981,
 'lease_3': 5677}

# Old

In [58]:
conn = psycopg2.connect(host="db", database="apartments", user="apartments", password="apartments")
cur = conn.cursor()

print('PostgreSQL database version:')
cur.execute('SELECT version()')

db_version = cur.fetchone()
print(db_version)

PostgreSQL database version:
('PostgreSQL 11.2 on x86_64-pc-linux-musl, compiled by gcc (Alpine 8.2.0) 8.2.0, 64-bit',)


In [14]:
cur.execute("ROLLBACK")

In [54]:
sql = """
INSERT INTO unit(
    apartment, 
    unit, 
    floorplan, 
    floor, 
    lease_3, 
    lease_4, 
    lease_5, 
    lease_6,
    lease_7,
    lease_8,
    lease_9,
    lease_10,
    lease_11,
    lease_12,
    created_on
) 
VALUES(
    %(apartment)s,
    %(unit)s,
    %(floorplan)s,
    %(floor)s,
    %(lease_3)s,
    %(lease_4)s,
    %(lease_5)s,
    %(lease_6)s,
    %(lease_7)s,
    %(lease_8)s,
    %(lease_9)s,
    %(lease_10)s,
    %(lease_11)s,
    %(lease_12)s,
    current_timestamp
)
"""
cur.executemany(sql, results)
conn.commit()

In [59]:
cur.execute('SELECT * from unit')

rows = cur.fetchall()
print(rows)

[(1, 'The Verge', '2380', 'The Cusp', -1, None, None, 2967, 2770, 4110, 2574, 2517, 2475, 2443, 2424, 2409, 2377, None, None, None, datetime.datetime(2019, 2, 19, 8, 13, 3, 426725)), (2, 'The Verge', '1466', 'The Cusp', -1, None, None, 5629, 2947, 2817, 2703, 2691, 2652, 2623, 2531, 2548, 2527, None, None, None, datetime.datetime(2019, 2, 19, 8, 13, 3, 426725)), (3, 'The Verge', '2639', 'The Cusp', -1, None, None, 5498, 2881, 2751, 2637, 2625, 2586, 2556, 2466, 2482, 2461, None, None, None, datetime.datetime(2019, 2, 19, 8, 13, 3, 426725)), (4, 'The Verge', '1367', 'Greenway', -1, None, None, 3175, 3003, 4384, 2788, 2720, 2649, 2650, 2625, 2605, 2531, None, None, None, datetime.datetime(2019, 2, 19, 8, 13, 3, 426725)), (5, 'The Verge', '2452', 'Greenway', -1, None, None, 3165, 2993, 4368, 2778, 2710, 2639, 2640, 2614, 2594, 2521, None, None, None, datetime.datetime(2019, 2, 19, 8, 13, 3, 426725)), (6, 'The Verge', '2466', 'Greenway', -1, None, None, 3120, 2947, 4296, 2732, 2664, 2593, 

In [56]:
cur.close()
conn.close()